In [0]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = spark.sql("SELECT * FROM classes.gold.df_exames").toPandas()

df['data_realizacao'] = pd.to_datetime(df['data_realizacao'])
df['mes_ano'] = df['data_realizacao'].dt.to_period('M').astype(str)

monthly = df.groupby('mes_ano').size().reset_index(name='quantidade')
monthly['mes_ano'] = pd.to_datetime(monthly['mes_ano'])

top_exames = df['nome_exame'].value_counts().head(15)
top_especialidade = df['especialidade_solicitante'].value_counts().head(15)

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=(
        "Evolução Mensal de Exames",
        "Top 15 Exames Mais Realizados",
        "Top 15 Especialidades Solicitantes"
    ),
    column_widths=[0.40, 0.30, 0.30],
    specs=[[{}, {"type": "bar"}, {"type": "bar"}]]
)

fig.add_trace(
    go.Scatter(x=monthly['mes_ano'], y=monthly['quantidade'],
               mode='lines+markers', line=dict(color='#2E91E5', width=3),
               marker=dict(size=6), name='Exames por mês'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=top_exames.values, y=top_exames.index, orientation='h',
           marker_color='#E24A33', showlegend=False),
    row=1, col=2
)

fig.add_trace(
    go.Bar(x=top_especialidade.values, y=top_especialidade.index, orientation='h',
           marker_color='#0B6623', showlegend=False),
    row=1, col=3
)

fig.update_layout(
    height=600,
    title_text="Análise de Exames Médicos (2020–2025)",
    title_x=0.5,
    showlegend=False,
    font=dict(size=12),
    margin=dict(l=60, r=60, t=90, b=60)
)

fig.update_xaxes(title_text="Data", row=1, col=1, tickangle=45)
fig.update_yaxes(title_text="Número de Exames", row=1, col=1)
fig.update_xaxes(title_text="Quantidade", row=1, col=2)
fig.update_xaxes(title_text="Quantidade", row=1, col=3)

fig.show()